In [ ]:
# 回到 FINRL/
%cd ..

In [ ]:
from AI_Trading.src import config
from AI_Trading.src.env_portfolio_allocation import *
from AI_Trading.src.evaluate import *
from AI_Trading.src import model_config
from AI_Trading.src.preprocess import *
from AI_Trading.src.testPortfolio import *
from AI_Trading.src.train import *
from AI_Trading.src.augmentation import *
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3

import pandas as pd
from warnings import simplefilter
simplefilter(action='ignore')
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline

In [ ]:
create_dir()

In [ ]:
# data preprocess
train,trade = preprocess(config.TRAIN_START_DATE[12], config.TRAIN_END_DATE[12], config.TEST_START_DATE[12], config.TEST_END_DATE[12])


In [ ]:
# training setting
model_index = 'test'
model_name = 'A2C'
exp = 'singleYear'

training_log_path = f'{config.LOG_PATH}training_log_{exp}.csv'
if os.path.exists(training_log_path):
     os.remove(training_log_path)

In [ ]:
env_kwargs = {
        "training_log_path": training_log_path,
        "hmax": 100, 
        "initial_amount": config.INITIAL_AMOUNT, 
        "transaction_cost_pct": 0.001, 
        "state_space": len(train.tic.unique()), 
        "stock_dim": len(train.tic.unique()), 
        "tech_indicator_list": config.INDICATORS, 
        "action_space": len(train.tic.unique()), 
        "reward_scaling": 1e-4
    }

env_train = portfolioAllocationEnv(df = train, is_test_set=False, **env_kwargs)
env_trade = portfolioAllocationEnv(df = trade, is_test_set=True, **env_kwargs)

# env_train = minReturnConcernEnv(df = train, is_test_set=False, **env_kwargs)
# env_trade = minReturnConcernEnv(df = trade, is_test_set=True, **env_kwargs)

# env_train = mddConcernEnv(df = train, is_test_set=False, **env_kwargs)
# env_trade = mddConcernEnv(df = trade, is_test_set=True, **env_kwargs)

# env_train = sharpeConcernEnv(df = train, is_test_set=False, **env_kwargs)
# env_trade = sharpeConcernEnv(df = trade, is_test_set=True, **env_kwargs)


In [ ]:
# train
trainPortfolioAllocation(exp, env_train, model_name, model_index)

In [ ]:
# load model
model_zip_path = f'{config.TRAINED_MODEL_PATH}/{exp}/{model_name}_{str(model_index)}.zip'
if model_name == 'A2C':
    model = A2C.load(model_zip_path)
elif model_name == 'PPO':
    model = PPO.load(model_zip_path)
elif model_name == 'DDPG':
    model = DDPG.load(model_zip_path)
elif model_name == 'TD3':
    model = TD3.load(model_zip_path)
elif model_name == 'SAC':
    model = SAC.load(model_zip_path)

In [ ]:
# test
df_daily_return, df_actions = test_portfolioAllocation(model, env_trade)
df_daily_return_index = df_daily_return.set_index('date')
df_daily_return_index.to_csv(f'{config.RESULTS_DIR}df_daily_return_{exp}_{model_index}.csv')
df_actions.to_csv(f'{config.RESULTS_DIR}df_action_{exp}_{model_index}.csv')

In [ ]:
# evaluate
print("==============DRL Strategy Stats===========")
DRL_returns, DRL_stats = getStats(df_daily_return)
DRL_stats

In [ ]:
print("==============Equal Weight Strategy Stats===========")
equalWeight_actions = getEqualWeightActions(trade)
df_equalWeight_return, df_portfolio_value = computeReturns(equalWeight_actions,trade, transCostRate= 0.001)
equalWeight_returns, equalWeight_stats = getStats(df_equalWeight_return)
equalWeight_stats

In [ ]:
print("==============Min-Variance Strategy Stats===========")
minVariance_actions = getMinVarianceActions(trade)
df_minVariance_return, df_minVariance_portfolio_value = computeReturns(minVariance_actions,trade, transCostRate=0.001)
minVariance_returns, minVariance_stats = getStats(df_minVariance_return)
minVariance_stats

In [ ]:
backtestPlot(DRL_returns, equalWeight_returns)

In [ ]:
print("==============All In Stock Strategy Stats===========")
all_stock_actions= getTicActions(trade, 'VTI')
df_stock_return, df_stock_value = computeReturns(all_stock_actions,trade, transCostRate=0.001)
stock_returns, stock_stats = getStats(df_stock_return)
stock_stats

In [ ]:
print("==============All In Debt Strategy Stats===========")
all_debt_actions= getTicActions(trade, 'TLT')
df_debt_return, df_debt_value = computeReturns(all_debt_actions,trade, transCostRate=0.001)
debt_returns, debt_stats = getStats(df_debt_return)
debt_stats

In [ ]:
print("==============All In Reit Strategy Stats===========")
all_reit_actions= getTicActions(trade, 'VNQ')
df_reit_return, df_reit_value = computeReturns(all_reit_actions,trade, transCostRate=0.001)
reit_returns, reit_stats = getStats(df_reit_return)
reit_stats

In [ ]:
cumulativeReturnPlot(DRL_returns, equalWeight_returns, minVariance_returns, stock_returns, debt_returns, reit_returns)

In [ ]:
weightTrend_plot(df_actions)

In [ ]:

df_VTI = load_data('./AI_Trading/data/VTI.csv', 'VTI')
df_VNQ = load_data('./AI_Trading/data/VNQ.csv', 'VNQ')
df_TLT = load_data('./AI_Trading/data/TLT.csv', 'TLT')

In [ ]:
cumulativeReturnPlot_ETF(stock_returns, debt_returns, reit_returns)

In [ ]:
closePlot_ETF(df_VTI, df_TLT, df_VNQ)

In [ ]:
# evaluate multi strategy
base = pd.read_csv('./AI_Trading/results/df_daily_return_base_12.csv', index_col='date')
mdd = pd.read_csv('./AI_Trading/results/df_daily_return_mdd_12.csv', index_col='date')
sharpe = pd.read_csv('./AI_Trading/results/df_daily_return_sharpe_12.csv', index_col='date')
minReturn = pd.read_csv('./AI_Trading/results/df_daily_return_minReturn_12.csv', index_col='date')
cumulativeReturnPlot(base.daily_return, mdd.daily_return, minReturn.daily_return, stock_returns, debt_returns, reit_returns)

In [ ]:
training_log = pd.read_csv('/Users/shrshiunhuang/Documents/Project/FinRL/AI_Trading/log/base/training_log_base_3.csv')
training_log

In [ ]:
# time_ind = training_log.index
trace0_portfolio = go.Scatter(y =training_log['mdd'], mode = 'lines', name = 'mdd')
# trace1_portfolio = go.Scatter(x = time_ind, y = all_debt.close, mode = 'lines', name = 'debt ETF')
# trace2_portfolio = go.Scatter(x = time_ind, y = all_reit.close, mode = 'lines', name = 'reit ETF')

fig = go.Figure()
fig.add_trace(trace0_portfolio)
# fig.add_trace(trace1_portfolio)
# fig.add_trace(trace2_portfolio)
fig.update_layout(
    legend=dict(
        x=0,
        y=1,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=10,
            color="black"
        ),
        bgcolor="White",
        bordercolor="white",
        borderwidth=2
        
    ),
)
fig.update_layout(title={
        #'text': "Cumulative Return using FinRL",
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
#with Transaction cost
fig.update_layout(
    paper_bgcolor='rgba(1,1,0,0)',
    plot_bgcolor='rgba(1, 1, 0, 0)',
    #xaxis_title="Date",
    yaxis_title="Close Price",
# xaxis={'type': 'date',
#     'tick0': time_ind[0],
#         'tickmode': 'linear',
#     'dtick': 86400000.0 *1000}

)
fig.update_xaxes(showline=True,linecolor='black',showgrid=True, gridwidth=1, gridcolor='LightSteelBlue',mirror=True)
fig.update_yaxes(showline=True,linecolor='black',showgrid=True, gridwidth=1, gridcolor='LightSteelBlue',mirror=True)
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='LightSteelBlue')

fig.show()